# Building a yield curve using US Treasury constant maturity data

We learned how to build curves via zero yields from the US Treasury data. Now we are going to pull prices for all maturities from [quandl](https://www.quandl.com/data/FRED-Federal-Reserve-Economic-Data/documentation/data-organization), and use that data to build functions that can price zero coupon bonds of any maturity less than the longest dated treasury. First, though, we copy some of the code we used for boostrapping from Treasury bills.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import quandl

quandl.ApiConfig.api_key = 'FA6wt7Na6c5FdbqM96i4'
ust_df = quandl.get('USTREASURY/BILLRATES', start_date='2017-01-01', end_date='2017-12-31')
# Drop the columns we don't want
for col in list(ust_df):
    if (col.find('Coupon Equiv') == -1):
        ust_df.drop([col], axis=1, inplace=True)
ust_df.drop(['8 Wk Coupon Equiv'], axis=1, inplace=True)

I noticed that the quandl data seems to have fewer maturity points than the [St. Louis FRED data](https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yield). So let's try adding some points.

In [7]:
# get all the data
maturities = [2, 3, 5, 7, 10, 20, 30]
columns = ['FRED/DGS'+str(i) for i in maturities]
ust_cm_df = quandl.get(columns, start_date='2017-01-01', end_date='2017-12-31')
ust_cm_df.tail()

,FRED/DGS2 - Value,FRED/DGS3 - Value,FRED/DGS5 - Value,FRED/DGS7 - Value,FRED/DGS10 - Value,FRED/DGS20 - Value,FRED/DGS30 - Value
Date,,,,,,,
2017-12-22,1.91,2.01,2.26,2.40,2.48,2.68,2.83
2017-12-26,1.92,2.02,2.25,2.38,2.47,2.66,2.82
2017-12-27,1.89,1.99,2.22,2.34,2.42,2.59,2.75
2017-12-28,1.91,2.00,2.23,2.36,2.43,2.60,2.75
2017-12-29,1.89,1.98,2.20,2.33,2.40,2.58,2.74


Now, we already had a RateTermStructure object. However, it was built from zeroes. The FRED data is for constant maturity par bonds. We will need to modify our class to build the first part of the curve from zero coupon bills, and the rest from constant maturity bonds. 

For a par bond with coupon rate $c$, paid semi-anuually, maturing at integer-valued $m$, and zero discount rate $z(t)$, the discounted cash flows are

$1 = \frac{c}{2} (e^{-z(0.5)} + e^{-z(1)}+ e^{-z(1.5)}+...+e^{-z(m )}) + e^{-z(m)} $.

The whole point of this is to find the zero rate function, which we know for the Treasury bills. For the bonds, we will assume as before that the forward rate curve is piecewise flat between maturities. We can divide the cash flows into those for which the forward rate is known (say maturity at most $m^*$, and those for which we need to compute the forward rate $f$.

$1 = \frac{c}{2} (e^{-z(0.5)} + e^{-z(1)}+ e^{-z(1.5)}+...+e^{-z(m^*)}) + \frac{c}{2} (e^{-z(m^*) - 0.5f} + e^{-z(m^*) - f}+ ...+e^{-z(m^*) - (m-m^*)f}) + e^{-z(m^*) - (m-m^*)f} $.

We can gather up the coupons that don't involve $f$ into a constant $C$:

$C = \frac{c}{2} (e^{-z(0.5)} + e^{-z(1)}+ e^{-z(1.5)}+...+e^{-z(m^*)})$.

Denoting $x=e^{-f/2}$, we can rewrite our cash equation as

$1 =  C + \frac{c}{2} e^{-z(m^*)}(x + x^2 + ... + x^{2(m - m^*)})) + e^{-z(m^*)}x^{2(m - m^*)}$ 

which we can rearrange to give

$(\frac{2}{c}+1)x^{2(m - m^*)} + x^{2(m - m^*)-1}+...+ x^3 + x^2 + x^1 + \frac{2}{c}(C - 1) e^{z(m^*)} = 0$ 

Conveniently, numpy has an API for solving polynomial equations. Let's test it for solving $x^2+x - 12 = 0$:

In [11]:
import numpy as np
coeffs = [1, 1, -12]
print(np.roots(coeffs))

[-4.  3.]


OK, we can get roots. Let's solve a fake coupon problem to see if we get a reasonable answer.

In [68]:
m = 2
m_star = 1
c = 0.1
f = c # let's assume forward rate matches coupon,
# times of cash flows between m_star and m
times = np.arange(0.5, m_star + 0.1, 0.5)
print(times)
disc_flows = [c/2 * np.exp(-t * f ) for t in times]
print(disc_flows)
C_ = np.sum(disc_flows)
print(C_)
z_of_m_star = f * m_star
coeffs = [1] * (2 * (m - m_star)+1)
coeffs[0] = 2 / c + 1
coeffs[-1] = 2/ c * ( C_ - 1) * np.exp(z_of_m_star)
roots = np.roots(coeffs)
print(roots)
f = -2 * np.log(roots[-1])
print(f)
# test f
more_discounts = [ np.exp(-z_of_m_star - f * (t-m_star)) for t in np.arange(m_star + 0.5, m + 0.1, 0.5)]
print(more_discounts)

C_ + c/2 * np.sum(more_discounts) + more_discounts[-1]

[0.5 1. ]
[0.047561471225035706, 0.04524187090179798]
0.0928033421268337
[-1.00127106  0.95365201]
0.09491288819655205
[0.8629000218914274, 0.8229063397891383]


1.0000000000000002